In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import py3Dmol

from proteome import protein
from proteome.models.design.foldingdiff.modeling import FoldingDiffForStructureDesign
from proteome.models.design.proteinmpnn.modeling import ProteinMPNNForSequenceDesign
from proteome.models.folding.omegafold.modeling import OmegaFoldForFolding

/home/conradry71/micromamba/envs/proteome/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-30 17:39:09.380970: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 17:39:10.261571: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
structure_designer = FoldingDiffForStructureDesign()
sequence_designer = ProteinMPNNForSequenceDesign("ca_only_model-20", random_seed=37)
folder = OmegaFoldForFolding()

Using time embedding: GaussianFourierProjection()


In [17]:
structure = structure_designer.design_structure()
generated_pdb = protein.to_pdb(structure)

sampling loop time step: 100%|███████████████████████████████████████████| 1000/1000 [00:14<00:00, 71.06it/s]


In [18]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(generated_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [19]:
sequence, score = sequence_designer.design_sequence(protein.to_ca_only_protein(structure))
print(f"Sequence: {sequence} with global_score {score}")

Sequence: LLALFLPPGAFVVPGLGPLGGLLGLVGGGGPGPLLPGGGAVPLLPLLLALLLLLGAPAPVLAALAAALAGAGLGGLGQVGLPLGLLGVFAGGAGAGGGAALLALLAALAGLLAARGLPLPGLRLRPRL with global_score 1.061846137046814


In [20]:
predicted_protein, confidence = folder.fold(sequence)
result_pdb = protein.to_pdb(predicted_protein)

In [21]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(result_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol